# Precomputation of open set performances

Used in Notebook 25 on Manuscript Section 1.

Scripted version in script_14*.py.

In [8]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from itertools import product


import torch
import torch.nn as nn
import torch.nn.functional as F


from NegativeClassOptimization import utils, preprocessing, ml
from NegativeClassOptimization import config, datasets

In [9]:
experiment_ids = ["11", "13", "14"]
df = utils.MLFlowTaskAPI.mlflow_results_as_dataframe(experiment_ids, run_name="dev-v0.1.2-3-with-replicates", classify_tasks=True)

KeyError in `mlflow_record_data`.
KeyError in `mlflow_record_data`.
KeyError in `mlflow_record_data`.
KeyError in `mlflow_record_data`.
KeyError in `mlflow_record_data`.
KeyError in `mlflow_record_data`.
KeyError in `mlflow_record_data`.
KeyError in `mlflow_record_data`.
KeyError in `mlflow_record_data`.
KeyError in `mlflow_record_data`.


In [10]:
print(df.columns)
print(df.shape)
df

Index(['sample_train', 'load_from_miniabsolut_split_seed', 'optimizer_type',
       'ag_neg', 'batch_size', 'epochs', 'num_hidden_units', 'N_closed',
       'N_train', 'split_id', 'ag_pos', 'swa', 'shuffle_antigen_labels',
       'input_dim', 'learning_rate', 'weight_decay', 'momentum', 'seed_id',
       'load_from_miniabsolut', 'acc_closed', 'train_loss', 'roc_auc_closed',
       'f1_closed', 'avg_precision_closed', 'precision_closed', 'test_loss',
       'recall_closed', 'mlflow.source.git.commit', 'mlflow.note.content',
       'mlflow.source.type', 'mlflow.runName', 'mlflow.source.name',
       'mlflow.log-model.history', 'mlflow.user', 'experiment', 'run_id',
       'task', 'split_seed'],
      dtype='object')
(1200, 38)


,sample_train,load_from_miniabsolut_split_seed,optimizer_type,ag_neg,batch_size,epochs,num_hidden_units,N_closed,N_train,split_id,...,mlflow.note.content,mlflow.source.type,mlflow.runName,mlflow.source.name,mlflow.log-model.history,mlflow.user,experiment,run_id,task,split_seed
0,None,4,Adam,1H0D,64,50,10,10000,30000,0,...,1OB1 vs 1H0D,LOCAL,dev-v0.1.2-3-with-replicates,scripts/script_12a_train_SN10_clean.py,"[{""run_id"": ""45e7f26f9d494b7cb2302f6f13c4bf55""...",eugen,11,45e7f26f9d494b7cb2302f6f13c4bf55,1v1,4
1,None,4,Adam,3VRL,64,50,10,10000,30000,0,...,1OB1 vs 3VRL,LOCAL,dev-v0.1.2-3-with-replicates,scripts/script_12a_train_SN10_clean.py,"[{""run_id"": ""32f8fd48823c40c5bba365c9243e02de""...",eugen,11,32f8fd48823c40c5bba365c9243e02de,1v1,4
2,None,4,Adam,1FBI,64,50,10,10000,30000,0,...,1OB1 vs 1FBI,LOCAL,dev-v0.1.2-3-with-replicates,scripts/script_12a_train_SN10_clean.py,"[{""run_id"": ""4f0eb2a80be645e3b6852c649537a693""...",eugen,11,4f0eb2a80be645e3b6852c649537a693,1v1,4
3,None,4,Adam,1WEJ,64,50,10,10000,30000,0,...,1OB1 vs 1WEJ,LOCAL,dev-v0.1.2-3-with-replicates,scripts/script_12a_train_SN10_clean.py,"[{""run_id"": ""6c6537db1ea246da8ab39ab5b5a8749b""...",eugen,11,6c6537db1ea246da8ab39ab5b5a8749b,1v1,4
4,None,4,Adam,1NSN,64,50,10,10000,30000,0,...,1OB1 vs 1NSN,LOCAL,dev-v0.1.2-3-with-replicates,scripts/script_12a_train_SN10_clean.py,"[{""run_id"": ""87a555c5f8fb4a92b497369e3113f76b""...",eugen,11,87a555c5f8fb4a92b497369e3113f76b,1v1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,None,None,Adam,1FBI_95low,64,50,10,10000,30000,0,...,1FBI_high vs 1FBI_95low,LOCAL,dev-v0.1.2-3-with-replicates,scripts/script_12d_train_SN10_clean_high_loose...,"[{""run_id"": ""f0ab3615846141619a6e7e8870541f65""...",eugen,14,f0ab3615846141619a6e7e8870541f65,high_vs_95low,42
196,None,None,Adam,1OB1_looser,64,50,10,10000,30000,0,...,1OB1_high vs 1OB1_looser,LOCAL,dev-v0.1.2-3-with-replicates,scripts/script_12d_train_SN10_clean_high_loose...,"[{""run_id"": ""c7fa79fc49f94647a5bd47ba5de0fefa""...",eugen,14,c7fa79fc49f94647a5bd47ba5de0fefa,high_vs_looser,42
197,None,None,Adam,2YPV_95low,64,50,10,10000,30000,0,...,2YPV_high vs 2YPV_95low,LOCAL,dev-v0.1.2-3-with-replicates,scripts/script_12d_train_SN10_clean_high_loose...,"[{""run_id"": ""ac8861eb8c9a446c9d20408d034be3ce""...",eugen,14,ac8861eb8c9a446c9d20408d034be3ce,high_vs_95low,42
198,None,None,Adam,3VRL_looser,64,50,10,10000,30000,0,...,3VRL_high vs 3VRL_looser,LOCAL,dev-v0.1.2-3-with-replicates,scripts/script_12d_train_SN10_clean_high_loose...,"[{""run_id"": ""c42fd3c99dcd44df971f9209e3dfad1c""...",eugen,14,c42fd3c99dcd44df971f9209e3dfad1c,high_vs_looser,42


In [11]:
def evaluate_model(
    model: torch.nn.Module,
    test_dataset: pd.DataFrame,
    ):
    """
    Evaluates a model on a test dataset.
    """
    with torch.no_grad():
        test_dataset = preprocessing.onehot_encode_df(test_dataset)
        X = np.stack(test_dataset["Slide_onehot"])
        X = torch.from_numpy(X).float()
        y_pred = model(X).round().detach().numpy().reshape(-1)
        y_true = test_dataset["binds_a_pos_ag"].values
        metrics = ml.compute_binary_metrics(y_pred, y_true)
    return metrics

In [12]:
## Generate valid seed_id and split_id combinations
## According to hard-coded logic in script_12*.py
seed_split_ids = []
for seed in [0, 1, 2, 3]:
    split_id_default = 42
    seed_split_ids.append((seed, split_id_default))
for split_id in [0, 1, 2, 3, 4]:
    seed_id_default = 0
    seed_split_ids.append((seed_id_default, split_id))

## Generate valid task pairings
task_types_for_openset = [
    datasets.ClassificationTaskType.ONE_VS_NINE,
    datasets.ClassificationTaskType.HIGH_VS_95LOW,
    datasets.ClassificationTaskType.HIGH_VS_LOOSER,
]
task_type_combinations = list(product(task_types_for_openset, task_types_for_openset))

## Load data
loader = datasets.FrozenMiniAbsolutMLLoader(data_dir=Path("../data/Frozen_MiniAbsolut_ML/"))

## Compute
records = []
for ag in config.ANTIGENS:
    for seed_id, split_id in seed_split_ids:
        for task_type_1, task_type_2 in task_type_combinations:                
            task_1 = datasets.ClassificationTask(
                task_type=task_type_1,
                ag_pos=ag,
                ag_neg="auto",
                seed_id=seed_id,
                split_id=split_id,
            )
            task_2 = datasets.ClassificationTask(
                task_type=task_type_2,
                ag_pos=ag,
                ag_neg="auto",
                seed_id=seed_id,
                split_id=split_id,
            )
            loader.load(task_1)
            loader.load(task_2)

            model = task_1.model
            test_dataset = task_2.test_dataset

            metrics = evaluate_model(model, test_dataset)
            records.append({
                "task_1": str(task_1),
                "task_2": str(task_2),
                **metrics,
            })

df_open = pd.DataFrame(records)

AttributeError: module 'NegativeClassOptimization.datasets' has no attribute 'ClassificationTaskType'